<a href="https://colab.research.google.com/github/zuzannazak/SentimentAnalysisInPolish/blob/main/Sentiment_analysis_in_Polish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Combining multiple files
**Input:** Csv files relating to 1 profile gathered from https://colab.research.google.com/drive/1P0mf1jhhYLqlDVN2eqC2HjbWeEg-6dmJ#scrollTo=QhTw_07h092i, uploaded to Colab

**Output:** One CSV file

In [ ]:
import os
import glob
import pandas as pd
os.chdir("/content")

In [ ]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [ ]:
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

#Sentiment analysis
**Input:** Prepared csv files gathered from https://colab.research.google.com/drive/1P0mf1jhhYLqlDVN2eqC2HjbWeEg-6dmJ#scrollTo=QhTw_07h092i and combined per social media account

**Output:** sentiment assigned to each tweet

In [ ]:
pip install -i https://pypi.clarin-pl.eu/ lpmn_client==1.4.1


Looking in indexes: https://pypi.clarin-pl.eu/


In [ ]:
 pip show lpmn_client

Name: lpmn-client
Version: 1.4.1
Summary: Client for writing LPMN queries.
Home-page: UNKNOWN
Author: Szymon Ciombor, Mateusz Gniewkowski
Author-email: Szymon.Ciombor@pwr.edu.pl, mateusz.gniewkowski@pwr.edu.pl
License: UNKNOWN
Location: /usr/local/lib/python3.7/dist-packages
Requires: requests, pyyaml
Required-by: 


#Sentiment analysis from csv file


In [ ]:
import pandas as pd
import numpy as np

import os

from lpmn_client import download_file, upload_file
from lpmn_client import Task

from google.colab import drive
drive.mount('/content/drive')

import time

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/SearchTweetPaczkomatyPL('2022-01-22T07_05_13.009Z',).csv")
df['sentiment'] = None
df = df.drop(columns=['Unnamed: 0'])

def multiLineHandling(inputString):
  return inputString.replace("\n", " ")

def deEmojify(inputString):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', inputString)

df["text"] = df["text"].apply(lambda s: multiLineHandling(s))

df["text_without_emojis"] = df["text"].apply(lambda s: deEmojify(s))


In [ ]:
df.head()

,text,in_reply_to_user_id,id,lang,author_id,referenced_tweets,reply_settings,public_metrics,created_at,source,conversation_id,geo,sentiment,text_without_emojis,sentiment_lpmn_wsd
0,@K_Kielczewski @PaczkomatyPL przyznacie się sa...,3.296933e+09,1482855036799623171,pl,819868801936293889,"[{'type': 'replied_to', 'id': '148278301475327...",everyone,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",2022-01-16T23:19:25.000Z,Twitter for iPhone,1482783014753271808,NaN,None,@K_Kielczewski @PaczkomatyPL przyznacie się sa...,-1
1,@r6_reggie @sg4rr_ @PaczkomatyPL XDD,1.385887e+18,1482710846056841224,und,1378155874913509377,"[{'type': 'replied_to', 'id': '148270418562800...",everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-01-16T13:46:27.000Z,Twitter for Android,1482630913137778692,NaN,None,@r6_reggie @sg4rr_ @PaczkomatyPL XDD,None
2,@Damian43211 @sg4rr_ @PaczkomatyPL Temu co pac...,1.378156e+18,1482704185628008451,pl,1385886999769403392,"[{'type': 'replied_to', 'id': '148267913136826...",everyone,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",2022-01-16T13:19:59.000Z,Twitter Web App,1482630913137778692,NaN,None,@Damian43211 @sg4rr_ @PaczkomatyPL Temu co pac...,0
3,@PaczkomatyPL A tam bot jest Jezus Maria 🖕,2.565559e+09,1482682057323077638,pl,3173102986,"[{'type': 'replied_to', 'id': '148268088370635...",everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-01-16T11:52:04.000Z,Twitter for Android,1482330351938805760,NaN,None,@PaczkomatyPL A tam bot jest Jezus Maria,0
4,@PaczkomatyPL @PaczkomatyPL I co... Teraz... ...,2.565559e+09,1482680022594957315,pl,3173102986,"[{'type': 'replied_to', 'id': '148234984800752...",everyone,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",2022-01-16T11:43:58.000Z,Twitter for Android,1482330351938805760,NaN,None,@PaczkomatyPL @PaczkomatyPL I co... Teraz... ...,0


In [ ]:
#szczegóły analizy sentymentu:
#    do ujednoznacznienia użyto wsd
#    W narzędziu ccl_emo ostateczny wydźwięk jest brany nie z pojedynczego pojęcia a z całego synsetu (synonimy) poprzez głosowanie większościowe.
#    Ta analiza sentymentu nie bierze pod uwagę emoji.


total=df.shape[0]
counter =0
df['sentiment_lpmn_wsd'] = None
for i, row in df.iterrows():
    unique_id = i
    val=(df['text_without_emojis'].values[i])
    with open('plik.txt', 'w') as f:
      f.write(val)
    file_id = upload_file("/content/plik.txt")
    t = Task('any2txt|wcrft2|wsd|ccl_emo({"lang":"polish"})|ccl_emo_stats({"lang":"polish", "split_paragraphs": false}) ')
    output_file_id = t.run(file_id)
    downloaded = download_file(output_file_id, "./out")

    try:
      df_sentiment=pd.read_csv(downloaded, compression='zip', header=0, sep=';')
      df_sentiment = df_sentiment.replace("None", 0)
      df_sentiment = df_sentiment.replace("amb", 0)
      df_sentiment["Polarity"] = pd.to_numeric(df_sentiment["Polarity"])
      column_sum = df_sentiment["Polarity"].sum()
      df['sentiment_lpmn_wsd'].values[i]= column_sum
      #print(df['text'].values[i] + ": " + str(column_sum))
      print(str(counter) + " out of " + str(total))
      counter = counter +1
    except:
      df['sentiment_lpmn_wsd'].values[i]= None
      #print(df['text'].values[i] + ": no sentiment for value " + str(i) )
      print(str(counter) + " out of " + str(total))
      counter = counter +1

    
    time.sleep(5)

df.to_csv('Paczkomatysentyment.csv')
df.to_csv('/content/drive/MyDrive/STUDIA/Doktorskie/Praca doktorska/Dane/Sentiment assigned/Paczkomatysentyment.csv')

0 out of 1101
1 out of 1101
2 out of 1101
3 out of 1101
4 out of 1101
5 out of 1101
6 out of 1101
7 out of 1101
8 out of 1101
9 out of 1101
10 out of 1101
11 out of 1101
12 out of 1101
13 out of 1101
14 out of 1101
15 out of 1101
16 out of 1101
17 out of 1101
18 out of 1101
19 out of 1101
20 out of 1101
21 out of 1101
22 out of 1101
23 out of 1101
24 out of 1101
25 out of 1101
26 out of 1101
27 out of 1101
28 out of 1101
29 out of 1101
30 out of 1101
31 out of 1101
32 out of 1101
33 out of 1101
34 out of 1101
35 out of 1101
36 out of 1101
37 out of 1101
38 out of 1101
39 out of 1101
40 out of 1101
41 out of 1101
42 out of 1101
43 out of 1101
44 out of 1101
45 out of 1101
46 out of 1101
47 out of 1101
48 out of 1101
49 out of 1101
50 out of 1101
51 out of 1101
52 out of 1101
53 out of 1101
54 out of 1101
55 out of 1101
56 out of 1101
57 out of 1101
58 out of 1101
59 out of 1101
60 out of 1101
61 out of 1101
62 out of 1101
63 out of 1101
64 out of 1101
65 out of 1101
66 out of 1101
67 ou

KeyboardInterrupt: ignored

In [ ]:
#szczegóły analizy sentymentu:
#    Pominięto wsd
#    W narzędziu ccl_emo ostateczny wydźwięk jest brany nie z pojedynczego pojęcia a z całego synsetu (synonimy) poprzez głosowanie większościowe.
#    Ta analiza sentymentu nie bierze pod uwagę emoji.


total=df.shape[0]
counter =0
df['sentiment_lpmn_no_wsd'] = None
for i, row in df.iterrows():
    unique_id = i
    val=(df['text_without_emojis'].values[i])
    with open('plik.txt', 'w') as f:
      f.write(val)
    file_id = upload_file("/content/plik.txt")
    t = Task('any2txt|wcrft2|ccl_emo({"lang":"polish"})|ccl_emo_stats({"lang":"polish", "split_paragraphs": false}) ')
    output_file_id = t.run(file_id)
    downloaded = download_file(output_file_id, "./out")

    try:
      df_sentiment=pd.read_csv(downloaded, compression='zip', header=0, sep=';')
      df_sentiment = df_sentiment.replace("None", 0)
      df_sentiment = df_sentiment.replace("amb", 0)
      df_sentiment["Polarity"] = pd.to_numeric(df_sentiment["Polarity"])
      column_sum = df_sentiment["Polarity"].sum()
      df['sentiment_lpmn_no_wsd'].values[i]= column_sum
      #print(df['text'].values[i] + ": " + str(column_sum))
      print(str(counter) + " out of " + str(total))
      counter = counter +1
    except:
      df['sentiment_lpmn_no_wsd'].values[i]= None
      #print(df['text'].values[i] + ": no sentiment for value " + str(i) )
      print(str(counter) + " out of " + str(total))
      counter = counter +1

    
    time.sleep(5)

df.to_csv('Paczkomatysentyment.csv')
df.to_csv('/content/drive/MyDrive/STUDIA/Doktorskie/Praca doktorska/Dane/Sentiment assigned/Paczkomatysentyment.csv')

##Sentiment check per zdanie

In [ ]:
sentiment_to_check = df.at[1064,'text_without_emojis']

In [ ]:
sentiment_to_check = "@PaczkomatyPL ... a na dostarczenie paczki reklamujecie  2 dni ? To ku... jakiś żart. Nie mamy pańskiego płaszcza.... . Znacie to ?"

In [ ]:
val=(sentiment_to_check)
with open('plik.txt', 'w') as f:
  f.write(val)
file_id = upload_file("/content/plik.txt")
t = Task('any2txt|wcrft2|wsd|ccl_emo({"lang":"polish"})|ccl_emo_stats({"lang":"polish", "split_paragraphs": false}) ')
output_file_id = t.run(file_id)
downloaded = download_file(output_file_id, "./out")

try:
    check_sentiment=pd.read_csv(downloaded, compression='zip', header=0, sep=';')
    check_sentiment = check_sentiment.replace("None", 0)
    check_sentiment = check_sentiment.replace("amb", 0)
    check_sentiment["Polarity"] = pd.to_numeric(check_sentiment["Polarity"])
    column_sum = check_sentiment["Polarity"].sum()
    print(sentiment_to_check + ": " + str(column_sum))
except:
    print(sentiment_to_check + ": no sentiment!")


@PaczkomatyPL Pegasus PostPost? : no sentiment!


In [ ]:
df_sentiment

,Lemma,Syn_id,Polarity,Emotion_names,Emotion_valuations,Units in synset
0,korzystać,1377,0,NaN,NaN,czerpać_korzyści.1(38:cpos) dobrze_wychodzić.1...
1,konkurencja,1340,0,NaN,NaN,kompetycja.1(8:zdarz) wyścig.2(8:zdarz) współz...


In [ ]:
df

,source,conversation_id,text,created_at,lang,in_reply_to_user_id,reply_settings,author_id,id,referenced_tweets,public_metrics,geo,sentiment
0,Twitter for Android,1475050133461032963,"@PaczkomatyPL Pewnie, że tak 😊 Już wszystko za...",2021-12-26T10:40:33.000Z,pl,2.565559e+09,everyone,52522222,1475053917746696196,"[{'type': 'replied_to', 'id': '147505300392113...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,-2
1,Twitter for Android,1475050133461032963,"Pierwszy raz mam taką sytuację, że nie mogę od...",2021-12-26T10:25:31.000Z,pl,NaN,everyone,52522222,1475050133461032963,NaN,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN,None
2,Twitter Web App,1474893391817232390,@PaczkomatyPL \nJest jakiś problem po waszej s...,2021-12-26T00:02:41.000Z,pl,2.565559e+09,everyone,2799071567,1474893391817232390,NaN,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,None
3,Twitter for Android,1474623367227617285,@CicheMale @MeteoprognozaPL @ebonit_7 @Doppelm...,2021-12-25T22:04:57.000Z,pl,7.113024e+17,everyone,1376873115091881986,1474863762792493059,"[{'type': 'replied_to', 'id': '147462336722761...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,None
4,Twitter Web App,1474623367227617285,@CicheMale @ebonit_7 @DoppelmayrGmbH @_PistenB...,2021-12-25T11:42:00.000Z,pl,7.113024e+17,everyone,2651114600,1474706994225463301,"[{'type': 'replied_to', 'id': '147462336722761...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
936,Twitter for Android,1479235375117778946,@OLogistyczny @InPostPL1 Chyba znowu oznaczyli...,2022-01-06T23:43:47.000Z,pl,1.382288e+18,everyone,1227343711278944257,1479237288097943555,"[{'type': 'replied_to', 'id': '147923537511777...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,None
937,Twitter for Android,1478035206904950788,@PaczkomatyPL @RBrzoska Czyli mam czekać 30 dn...,2022-01-06T18:23:59.000Z,pl,8.590470e+17,everyone,859047002381967361,1479156810233700355,"[{'type': 'replied_to', 'id': '147908802222645...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,None
938,Twitter for iPad,1478833901510475785,@PaczkomatyPL @HERYTlERA A czy w piątek kurier...,2022-01-06T16:24:41.000Z,pl,2.565559e+09,everyone,291712315,1479126786923548672,"[{'type': 'replied_to', 'id': '147883390151047...","{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN,None
939,Twitter Web App,1478035206904950788,@PaczkomatyPL 2/2 do tego reklamacja przez str...,2022-01-06T13:50:39.000Z,pl,8.590470e+17,everyone,859047002381967361,1479088022226452480,"[{'type': 'replied_to', 'id': '147908719495022...","{'retweet_count': 0, 'reply_count': 2, 'like_c...",NaN,None


#Sentiment analysis from csv via zip

In [3]:
pip install -i https://pypi.clarin-pl.eu/ lpmn_client==1.4.3


Looking in indexes: https://pypi.clarin-pl.eu/


In [4]:
import pandas as pd
import numpy as np

import os

from lpmn_client import download_file, upload_file
from lpmn_client import Task

from google.colab import drive
drive.mount('/content/drive')

import time

import re

from zipfile import ZipFile


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
filepath = "/content/drive/MyDrive/STUDIA/Doktorskie/Praca doktorska/Dane/Search API/SearchTweetPaczkomatyPL('2022-01-22T07_05_13.009Z',).csv"

In [6]:
df = pd.read_csv(filepath)
df['sentiment'] = None
df = df.drop(columns=['Unnamed: 0'])

def multiLineHandling(inputString):
  return inputString.replace("\n", " ")

def deEmojify(inputString):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', inputString)

df["text"] = df["text"].apply(lambda s: multiLineHandling(s))

df["text_without_emojis"] = df["text"].apply(lambda s: deEmojify(s))


In [8]:
df['sentiment_lpmn_wsd'] = None
with ZipFile('ZipFolder.zip', 'w') as zipObj:
  for i, row in df.iterrows():
      unique_id = i
      val=(df['text_without_emojis'].values[i])
      with open(('text_without_emojis{track}.txt'.format(track=i)),'w') as f:
        f.write(val)
      zipObj.write(('text_without_emojis{track}.txt'.format(track=i)))



In [10]:
t = Task('any2txt|wcrft2|wsd|ccl_emo({"lang":"polish"})|ccl_emo_stats({"lang":"polish", "split_paragraphs": false}) ')
file_id = upload_file("/content/ZipFolder.zip")  # zip file with any documents, for examples docx files
output_file_id = t.run(file_id)
download_file(output_file_id, "./out")

'./out/7ab8553d-c956-4e7d-ab0c-d1827958875c.zip'

#Visualisation
Materials:

https://medium.com/@siddhardhan23/data-visualization-in-python-a90ddb706b23

https://towardsdatascience.com/5-python-gui-frameworks-to-create-desktop-web-and-even-mobile-apps-c25f1bcfb561

